# OpenAQ-fetch

OpenAQ-fetch collate real time air quality monitoring data from across the world. The program liberate real time data from myriads of public websites into open, resuable form towards addressing air pollution problems. 
1. Code base of the program is backed by Node.js javascript run-time environmet and follows ES6 programming standard.
1. Major components of javscript based program is as follows
![Components of Openaq-fetch](https://raw.githubusercontent.com/nishadhka/openaq-fetch/jupyter_documentation/docs/openaq_fetch.png)
1. Scripts grouped into adaptors does collection, formating and tranfer collected data into database storage. Adaptors has to be customzied to each specific websites to collect data. 
1. Developing and customizing adaptors could be simplified by documenting logic, functions behind the javascript code and by testing code snippets. Jupyter notebook gives facility to both execute and gives description of code single interface. Which can be used to document, try with adaptor code by excuting trial codes suitable for source website. 
1. Jupyter gives javascript kernels such as [this](https://github.com/n-riesco/ijavascript). However to execute OpenAQ codes which is based on ES6 needs babel kernal [jp-babel](https://github.com/n-riesco/jp-babel)
1. setting up the envirotmnet is as described in resepective project documentation, the notebook however has to be placed within the openaq folder to correctly catch with the libraries

# testing data_scritps 
1. Getting defra-locations based on file data_scripts/defra-locations.js

In [1]:
'use strict';

import cheerio from 'cheerio';
import { parallelLimit } from 'async';
var request = require('request');

function handleLink (url) {
  return function (done) {
    request(url, (err, res, body) => {
      if (err) {
        done(err);
      }
      let $ = cheerio.load(body);
      let city = $($('#tab_info').find('p').get(5)).html();
      city = city.split('</strong>')[1].trim();
      let coords = $($('#tab_info').find('p').get(8)).html();
      coords = coords.split('</strong>')[1].trim();
      coords = {
        latitude: Number(coords.split(',')[0]),
        longitude: Number(coords.split(',')[1])
      };

      done(null, {city: city, coordinates: coords});
    });
  };
}

request('http://uk-air.defra.gov.uk/latest/currentlevels', (err, res, body) => {
  if (err) {
    return console.error(err);
  }

  // Get links from main page
  console.info('Grabbing main page to get locations.');
  let $ = cheerio.load(body);
  let links = {};
  $('.current_levels_table').each((i, e) => {
    $('tr', $(e)).each((i, e) => {
      let link = $($(e).find('td a')).attr('href');
      let name = $($(e).find('td a')).html();
      if (link) {
        links[name] = handleLink(link.replace('../', 'http://uk-air.defra.gov.uk/'));
      }
    });
  });

  // Get info from each link
  console.info(`Grabbing data for ${Object.keys(links).length} locations.`);
  // links = {'Auchencorth Moss': links['Auchencorth Moss']};
  parallelLimit(links, 5, (err, results) => {
    console.info('Grabbing all data completed!');
    if (err) {
      return console.error(err);
    }

    console.info(results);
  });
});

Request {
  _events:
   { error: [Function: bound ],
     complete: [Function: bound ],
     pipe: [Function] },
  _eventsCount: 3,
  _maxListeners: undefined,
  uri:
   Url {
     protocol: 'http:',
     slashes: true,
     auth: null,
     host: 'uk-air.defra.gov.uk',
     port: 80,
     hostname: 'uk-air.defra.gov.uk',
     hash: null,
     search: null,
     query: null,
     pathname: '/latest/currentlevels',
     path: '/latest/currentlevels',
     href: 'http://uk-air.defra.gov.uk/latest/currentlevels' },
  callback: [Function],
  readable: true,
  writable: true,
  _qs:
   Querystring {
     request: [Circular],
     lib:
      { formats: [Object], parse: [Function], stringify: [Function] },
     useQuerystring: undefined,
     parseOptions: {},
     stringifyOptions: {} },
  _auth:
   Auth {
     request: [Circular],
     hasAuth: false,
     sentAuth: false,
     bearerToken: null,
     user: null,
     pass: null },
  _oauth: OAuth { request: [Circular], params: null },
  _m

Grabbing main page to get locations.
Grabbing data for 168 locations.
Grabbing all data completed!
{ 'Bush Estate':
   { city: 'Bush Estate',
     coordinates: { latitude: 55.862281, longitude: -3.205782 } },
  'Dumbarton Roadside':
   { city: 'Dumbarton',
     coordinates: { latitude: 55.943197, longitude: -4.55973 } },
  'Edinburgh St Leonards':
   { city: 'Edingurgh',
     coordinates: { latitude: 55.945589, longitude: -3.182186 } },
  'Edinburgh Nicolson Street':
   { city: 'Footpath outside of 101 Nicolson Street Edinburgh  EH8 9BY',
     coordinates: { latitude: 55.94476, longitude: -3.183991 } },
  'Auchencorth Moss':
   { city: 'Auchencorth',
     coordinates: { latitude: 55.79216, longitude: -3.2429 } },
  'Glasgow Great Western Road':
   { city: 'Not available',
     coordinates: { latitude: 55.872038, longitude: -4.270936 } },
  'Glasgow Kerbside':
   { city: 'Glasgow',
     coordinates: { latitude: 55.85917, longitude: -4.258889 } },
  'Glasgow High Street':
   { city: 'Gla

## testing csv parsing
1. using  arpae-locations.js

In [23]:
'use strict';

/*
   A script for generating a locations mapping
   object from the ARPAE parameters source.

 */

import { default as parse } from 'csv-parse/lib/sync';
import { uniqBy } from 'lodash';
var request_te = require('request');

const csvUrl_ev = 'https://docs.google.com/spreadsheets/d/1GlY3Pu9GDpLDk8Spl9yV1wjCRPvOI1m7BFxumfcuGcE/export?format=csv';

request_te(csvUrl_ev, (err, res, data) => {
  if (err) {
    console.log(err);
  }

  const parsed = parse(data, { columns: true });
  let locations = {};
  uniqBy(parsed, 'Cod_staz').forEach((o) => {
    locations[o.Cod_staz] = {
      station: o.Stazione,
      comune: o.COMUNE,
      coordinates: {
        latitude: Number(o.Lat),
        longitude: Number(o.Lon)
      }
    };
  });
  console.log(locations);
});

Request {
  _events:
   { error: [Function: bound ],
     complete: [Function: bound ],
     pipe: [Function] },
  _eventsCount: 3,
  _maxListeners: undefined,
  uri:
   Url {
     protocol: 'https:',
     slashes: true,
     auth: null,
     host: 'docs.google.com',
     port: 443,
     hostname: 'docs.google.com',
     hash: null,
     search: '?format=csv',
     query: 'format=csv',
     pathname:
      '/spreadsheets/d/1GlY3Pu9GDpLDk8Spl9yV1wjCRPvOI1m7BFxumfcuGcE/export',
     path:
      '/spreadsheets/d/1GlY3Pu9GDpLDk8Spl9yV1wjCRPvOI1m7BFxumfcuGcE/export?format=csv',
     href:
      'https://docs.google.com/spreadsheets/d/1GlY3Pu9GDpLDk8Spl9yV1wjCRPvOI1m7BFxumfcuGcE/export?format=csv' },
  callback: [Function],
  readable: true,
  writable: true,
  _qs:
   Querystring {
     request: [Circular],
     lib:
      { formats: [Object], parse: [Function], stringify: [Function] },
     useQuerystring: undefined,
     parseOptions: {},
     stringifyOptions: {} },
  _auth:
   Auth {
  

{ '2000003':
   { station: 'CITTADELLA',
     comune: 'PARMA',
     coordinates: { latitude: 44.7924, longitude: 10.331 } },
  '2000004':
   { station: 'MONTEBELLO',
     comune: 'PARMA',
     coordinates: { latitude: 44.7866, longitude: 10.3365 } },
  '2000214':
   { station: 'BADIA',
     comune: 'LANGHIRANO',
     coordinates: { latitude: 44.6582, longitude: 10.2894 } },
  '2000219':
   { station: 'SARAGAT',
     comune: 'COLORNO',
     coordinates: { latitude: 44.926, longitude: 10.3719 } },
  '2000229':
   { station: 'MALCANTONE',
     comune: 'MEZZANI',
     coordinates: { latitude: 44.8909, longitude: 10.3996 } },
  '2000230':
   { station: 'BOGOLESE',
     comune: 'SORBOLO',
     coordinates: { latitude: 44.8291, longitude: 10.3915 } },
  '2000232':
   { station: 'PARADIGNA',
     comune: 'PARMA',
     coordinates: { latitude: 44.8413, longitude: 10.3446 } },
  '3000001':
   { station: 'CASTELLARANO',
     comune: 'CASTELLARANO',
     coordinates: { latitude: 44.5162, longitude